In [1]:
import numpy as np
from scipy.optimize import linprog

# Первая неделя

In [5]:
# Вектор затрат на транспортировку (не путать с матрицей). 
# Первые 7 псевдостолбиков - клиенты, последние 3 - склады.
# Псевдострочки - заводы.

ship_prices = np.array([84, 36, 42, 81, 63, 60, 66, 78, 15, 42, 
                                    63, 48, 33, 24, 33, 21, 33, 33, 60, 60,
                                    63, 18, 33, 66, 45, 45, 51, 60, 9, 24,
                                    39, 33, 57, 63, 42, 51, 45, 51, 45, 15,
                                    30, 21, 42, 42, 24, 33, 24, 33, 39, 12])

In [6]:
factories = 5 #число заводов

variables = len(ship_prices) #число переменных 
print(variables)

points = int(variables/factories) #число пунктов доставки (куда доставить)
print(points)

50
10


In [7]:
reserves = np.array([290, 165, 235, 255, 105]) #резервы (запасы)
demand = np.array([90, 65, 45, 75, 95, 100, 75, 150, 150, 205]) # спрос (в том числе и на доставку на склады)

In [8]:
# Функция для создания ограничений на запасы:

def matrix_of_reserves_limitations(factories, variables):
    
    res = np.zeros(shape=(factories, variables), dtype=int) #Матрица нулей размера заводы*переменные
    count = int(variables / factories)  #Локальная переменная - число пунктов доставки
    
    for index in range(factories): #Для чисел от нуля до количества заводов
        begin = count * index   #стартовать вычисления будем с этой точки (в нашей задаче begin = (0, 10, 20, 30, 40)) .
        temp = np.zeros(variables) #Временная переменная - нулевой вектор длиной variables,для которого
        temp[begin:(begin+count)] = 1 #срез от стартовой точки до заданной мы заменяем единицами
        res[index] = temp #и записываем полученное изменение в res.
           
    return res

In [9]:
# Результат выполнения данной функции записываем в переменную.
limitations_to_reserves=matrix_of_reserves_limitations(factories, variables)

#Любуемся.
limitations_to_reserves

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]])

In [10]:
# Функция для создания ограничений на заказы:

def matrix_of_demand_limitations(factories, variables):
    
    count = int(variables / factories) #Локальная переменная - число пунктов доставки
    res = np.zeros(shape=(count, variables), dtype=int) #Матрица нулей размера пункты*переменные      
    begin = 0 #Точка отсчета начинается в нуле.
    
    while begin < count: #Пока мы не проделаем число итераций, равное count
        
        for index in range(begin, count): #Для индексов от точки отсчета до count
            temp = np.zeros(variables, dtype=int) #Временная переменная - вектор нулей длины variables
            indexTemp = 0 #Временный индекс начинается с нуля.
            
            for index2 in range(begin,variables,count): #Для других индексов от точки отсчета до количества переменных с шагом count
                temp[indexTemp+begin] = 1 #Заменяем временную переменную на позиции "indexTemp+begin" на единицу.
                indexTemp += count #Прибавляем временный счетчик (в нашей задаче на 10)
                
            begin+=1 #Во внешнем цикле увеличиваем точку отсчета на единицу
            res[index] = temp #Записываем результат в res
            
            
    return res

In [11]:
# Результат выполнения данной функции записываем в переменную.
limitations_to_demand=matrix_of_demand_limitations(factories, variables)

#Любуемся.
limitations_to_demand

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0

In [13]:
#Решаю симплекс-методом. Он медленне остальных и относительно старый, но зато целочисленный.
first_week=linprog(c=ship_prices, 
              A_ub=limitations_to_reserves, b_ub=reserves,
              A_eq=limitations_to_demand, b_eq=demand,method='simplex')
first_week

     con: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
     fun: 31815.0
 message: 'Optimization terminated successfully.'
     nit: 45
   slack: array([0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([  0.,   0.,  45.,   0.,   0.,  95.,   0.,   0., 150.,   0.,   0.,
         0.,   0.,  75.,   0.,   0.,   0.,  90.,   0.,   0.,   0.,  65.,
         0.,   0.,  95.,   5.,  30.,   0.,   0.,  40.,  90.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0., 165.,   0.,   0.,   0.,   0.,
         0.,   0.,  45.,  60.,   0.,   0.])

In [14]:
#Более удобное представление результата доставок в первую неделю работы.
np.reshape(first_week['x'],(factories,points))

array([[  0.,   0.,  45.,   0.,   0.,  95.,   0.,   0., 150.,   0.],
       [  0.,   0.,   0.,  75.,   0.,   0.,   0.,  90.,   0.,   0.],
       [  0.,  65.,   0.,   0.,  95.,   5.,  30.,   0.,   0.,  40.],
       [ 90.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 165.],
       [  0.,   0.,   0.,   0.,   0.,   0.,  45.,  60.,   0.,   0.]])

# Вторая неделя

In [15]:
# Вектор затрат на транспортировку. 
# Псевдостолбики - клиенты.
# Псевдострочки - склады.

warehouse_prices = np.array([27, 57, 63, 21, 30, 39, 24,
                                                69, 21, 27, 66, 48, 45, 51,
                                                42, 18, 42, 54, 33, 39, 36])

In [16]:
warehouses = 3

second_variables = len(warehouse_prices) #число переменных 
print(second_variables)

second_points = int(second_variables/warehouses) #число пунктов доставки (куда доставить)
print(second_points)

21
7


In [17]:
starage_reserves = np.array([150,150,205]) #складские запасы
second_demand = np.array([55, 45, 70, 75, 40, 35, 185]) # спрос второй недели

In [18]:
# Результат выполнения данной функции записываем в переменную.
limit_storage=matrix_of_reserves_limitations(warehouses, second_variables)

#Любуемся.
limit_storage

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])

In [19]:
# Результат выполнения данной функции записываем в переменную.
limitat_demand=matrix_of_demand_limitations(warehouses, second_variables)

#Любуемся.
limitat_demand

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]])

In [20]:
#Решаю симплекс-методом. Он медленне остальных и относительно старый, но зато целочисленный.
second_week=linprog(c=warehouse_prices, 
              A_ub=limit_storage, b_ub=starage_reserves,
              A_eq=limitat_demand, b_eq=second_demand,method='simplex')
second_week

     con: array([0., 0., 0., 0., 0., 0., 0.])
     fun: 15210.0
 message: 'Optimization terminated successfully.'
     nit: 27
   slack: array([0., 0., 0.])
  status: 0
 success: True
       x: array([ 55.,   0.,   0.,  75.,   0.,   0.,  20.,   0.,  45.,  70.,   0.,
         0.,  35.,   0.,   0.,   0.,   0.,   0.,  40.,   0., 165.])

In [21]:
#Более удобное представление результата доставок в первую неделю работы.
np.reshape(second_week['x'],(warehouses,second_points))

array([[ 55.,   0.,   0.,  75.,   0.,   0.,  20.],
       [  0.,  45.,  70.,   0.,   0.,  35.,   0.],
       [  0.,   0.,   0.,   0.,  40.,   0., 165.]])

Общий результат оптимизации

In [22]:
good_score = second_week['fun']+first_week['fun']
good_score

47025.0

# Что было бы без "тотальной" оптимизации

Доставка завод-клиент

In [176]:
# Вектор затрат на транспортировку с завода до клиента. 
# Псевдостолбикb - клиенты.
# Псевдострочки - заводы.

ship_prices_f_c = np.array([84, 36, 42, 81, 63, 60, 66, 
                                    63, 48, 33, 24, 33, 21, 33, 
                                    63, 18, 33, 66, 45, 45, 51, 
                                    39, 33, 57, 63, 42, 51, 45,
                                    30, 21, 42, 42, 24, 33, 24])

In [179]:
factories = 5 #число заводов
clients = 7

variables_f_c = len(ship_prices_f_c) #число переменных 
print(variables_f_c)

35


In [180]:
reserves = np.array([290, 165, 235, 255, 105]) #резервы (запасы)
demand_f_c_1 = np.array([90, 65, 45, 75, 95, 100, 75]) # спрос 

In [181]:
# Результат выполнения данной функции записываем в переменную.
limitations_to_reserves_f_c=matrix_of_reserves_limitations(factories, variables_f_c)

#Любуемся.
limitations_to_reserves_f_c

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])

In [182]:
# Результат выполнения данной функции записываем в переменную.
limitations_to_demand_f_c=matrix_of_demand_limitations(factories, variables_f_c)

#Любуемся.
limitations_to_demand_f_c

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]])

In [184]:
#Решаю симплекс-методом. Он медленне остальных и относительно старый, но зато целочисленный.
deliver_f_c=linprog(c=ship_prices_f_c, 
              A_ub=limitations_to_reserves_f_c, b_ub=reserves,
              A_eq=limitations_to_demand_f_c, b_eq=demand_f_c_1,method='simplex')
deliver_f_c

     con: array([0., 0., 0., 0., 0., 0., 0.])
     fun: 15555.0
 message: 'Optimization terminated successfully.'
     nit: 42
   slack: array([290.,   0., 115., 100.,   0.])
  status: 0
 success: True
       x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 75.,  0., 90.,
        0.,  0., 65., 45.,  0.,  0., 10.,  0., 90.,  0.,  0.,  0., 65.,
        0.,  0.,  0.,  0.,  0.,  0., 30.,  0., 75.])

In [186]:
#Более удобное представление результата доставок с завода клиенту.
np.reshape(deliver_f_c['x'],(factories,clients))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 75.,  0., 90.,  0.],
       [ 0., 65., 45.,  0.,  0., 10.,  0.],
       [90.,  0.,  0.,  0., 65.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 30.,  0., 75.]])

Доставка завод-склад

In [187]:
# Вектор затрат на транспортировку с завода до склада. 
# Псевдостолбики - склады.
# Псевдострочки - заводы.

ship_prices_f_w = np.array([78, 15, 42,
                                            33, 60, 60,
                                            60, 9, 24,
                                            51, 45, 15,
                                            33, 39, 12])

In [189]:
# Число складов
warehouses = 3

variables_f_w = len(ship_prices_f_w) #число переменных 
print(variables_f_w)

15


In [193]:
capacity = np.array([150,150, 205]) #вместимость складов

remains = deliver_f_c['slack'] #остатки нужно увезти на склады
remains

array([290.,   0., 115., 100.,   0.])

In [211]:
# Результат выполнения данной функции записываем в переменную. Ограничения 
limitations_to_reserves_f_w=matrix_of_reserves_limitations(factories, variables_f_w)

#Любуемся.
limitations_to_reserves_f_w

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

In [212]:
# Результат выполнения данной функции записываем в переменную.
limitations_to_remains=matrix_of_demand_limitations(factories, variables_f_w)

#Любуемся.
limitations_to_remains

array([[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]])

In [214]:
#Решаю симплекс-методом. Он медленне остальных и относительно старый, но зато целочисленный.
deliver_f_w=linprog(c=ship_prices_f_w, 
              A_ub=limitations_to_remains, b_ub=capacity,
              A_eq=limitations_to_reserves_f_w, b_eq=remains ,method='simplex')
deliver_f_w

     con: array([0., 0., 0., 0., 0.])
     fun: 17790.0
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([0., 0., 0.])
  status: 0
 success: True
       x: array([  0., 150., 140.,   0.,   0.,   0.,  50.,   0.,  65., 100.,   0.,
         0.,   0.,   0.,   0.])

In [215]:
#Более удобное представление результата доставок с завода на склады.
np.reshape(deliver_f_w['x'],(factories,warehouses))

array([[  0., 150., 140.],
       [  0.,   0.,   0.],
       [ 50.,   0.,  65.],
       [100.,   0.,   0.],
       [  0.,   0.,   0.]])

Доставка склад - клиент не изменилась.

# Итого

In [218]:
#Считаем затраты
bad_score = second_week['fun'] + deliver_f_c['fun'] +deliver_f_w['fun']
bad_score

48555.0

In [229]:
#Считаем экономию 
save = bad_score - good_score
print('Мы сэкономили', save, 'у.е.,')

improve = 100-round(good_score/bad_score*100)
print('(или', improve, '%) расходов при "тотальной" оптимизации')

Мы сэкономили 1530.0 у.е.,
(или 3.0 %) расходов при "тотальной" оптимизации
